In [ ]:
import asyncio
import logging
import sys

from lsst.ts import salobj

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
await asyncio.sleep(10)

In [ ]:
CCCamera = salobj.Remote(domain, 'CCCamera')
await CCCamera.start_task

In [ ]:
cal_images = {
    "day_obs": 20210401,
    "bias": {"start": 7, "end": 16},
    "dark": {"start": 17, "end": 26},
    "flat": {"start": 27, "end": 36},
    "daq_folder": "comcam-cal"
}
ptc_images = {
    "day_obs": 20210401,
    "ptc": {"start": 42, "end": 61},
    "daq_folder": "comcam-ptc"
}
file_stem = "CC_O_{0}_{1:06}"

In [ ]:
def make_file_list(idict, key):
    image_files = []
    for i in range(idict[key]["start"], idict[key]["end"] + 1):
        image_files.append(file_stem.format(idict["day_obs"], i))
    return image_files

In [ ]:
# Create bias playlist
ack = await CCCamera.cmd_playlist.set_start(playlist="comcam-bias.playlist",
                                            folder=cal_images["daq_folder"],
                                            images=":".join(make_file_list(cal_images, "bias")),
                                            timeout=60)
print(ack)

In [ ]:
# Create dark playlist
ack = await CCCamera.cmd_playlist.set_start(playlist="comcam-dark.playlist",
                                            folder=cal_images["daq_folder"],
                                            images=":".join(make_file_list(cal_images, "dark")),
                                            timeout=60)
print(ack)

In [ ]:
# Create flat playlist
ack = await CCCamera.cmd_playlist.set_start(playlist="comcam-flat.playlist",
                                            folder=cal_images["daq_folder"],
                                            images=":".join(make_file_list(cal_images, "flat")),
                                            timeout=60)
print(ack)

In [ ]:
# Create calibration playlist
biases = make_file_list(cal_images, "bias")
darks = make_file_list(cal_images, "dark")
flats = make_file_list(cal_images, "flat")
ack = await CCCamera.cmd_playlist.set_start(playlist="comcam-cal.playlist",
                                            folder=cal_images["daq_folder"],
                                            images=":".join(biases + darks + flats),
                                            timeout=60)
print(ack)

In [ ]:
# Create ptc playlist
ack = await CCCamera.cmd_playlist.set_start(playlist="comcam-ptc.playlist",
                                            folder=ptc_images["daq_folder"],
                                            images=":".join(make_file_list(ptc_images, "ptc")),
                                            timeout=60)
print(ack)